In [1]:
num_epochs = 10
batch_size = 32
best_accuracy = 50
#num_layers = 9

In [2]:
import torch
import torch.nn as nn
import numpy as np
import os
import pandas as pd
import dask.dataframe as dd
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from IPython.display import clear_output
import time

In [3]:
class MyDataset(Dataset):
    def __init__(self, file_path, chunk_size):
        self.file_path = file_path
        self.chunk_size = chunk_size
        
        self.columns = pd.read_csv(self.file_path, nrows=0).columns.str.strip().str.replace('\xa0', ' ').tolist()
        self.train_columns = [col for col in self.columns if col != 'if riichi']  # 排除 'if riichi' 欄位
        self.num_samples = sum(1 for _ in open(self.file_path, encoding='utf-8')) - 1  # 計算總樣本數
        
    def __getitem__(self, idx):
        chunk_start = idx // self.chunk_size * self.chunk_size
        df = pd.read_csv(self.file_path, skiprows=chunk_start + 1, nrows=self.chunk_size, header=None, encoding='utf-8')  # 跳過標題和之前的行

        df.columns = self.columns
        
        if 'if riichi' not in df.columns:
            raise KeyError(f"Chunk starting at row {chunk_start + 1} does not contain 'if riichi' column.")
        
            
        sample_idx = idx % self.chunk_size
        
        train_data = df[self.train_columns].iloc[sample_idx].values.astype("float32")
        train_data = train_data.reshape(34, 29)
        
        value_data = df['if riichi'].iloc[sample_idx]  # 提取 'if riichi' 欄位
        
        return torch.tensor(train_data, dtype=torch.float32), torch.tensor(value_data, dtype=torch.long)
        
    def __len__(self):
        return self.num_samples

In [4]:
"""
讀取csv
"""
file_path = 'E:/專題/data/2021/RiichiData.csv'
dataset = MyDataset(file_path, chunk_size=50000)  # 每次只加載 50000 行
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [5]:
class MyCNN(nn.Module):
    def __init__(self, num_layers = 10, input_features=34, input_channels=29, output_features=1):
        super(MyCNN, self).__init__()
        
        self.conv_layers = nn.ModuleList()

        self.conv_layers.append(nn.Conv1d(in_channels=input_channels, out_channels=256, kernel_size=3, padding=1))
        
        for _ in range(1, num_layers - 1):
            self.conv_layers.append(nn.Conv1d(in_channels=256, out_channels=256, kernel_size=3, padding=1))
        
        self.conv_layers.append(nn.Conv1d(in_channels=256, out_channels=1, kernel_size=1, padding=0))
        
        self.activation = nn.LeakyReLU(negative_slope=0.01)

        self.fc = nn.Linear(input_features, output_features)

        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = x.squeeze(1)
        x = x.permute(0, 2, 1)

        for i, conv in enumerate(self.conv_layers):
            if(i < len(self.conv_layers) - 1):
                x = self.activation(conv(x))
            else:
                x = conv(x)
                
        x = x.squeeze(1)
        x = self.fc(x)
        x = self.sigmoid(x)
        print("x.shape:",x.shape)
        
        return x

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [7]:
model = MyCNN().to(device)
loss_criterion = nn.BCELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.001)

In [ ]:
current_loss = 0

start_time = time.time()
last_saved_time = start_time

for epoch in range(num_epochs):
    for train_data, value_data in train_dataloader:
        torch.autograd.set_detect_anomaly(True)

        channel1_tensor = train_data.unsqueeze(1).to(device)
        value_data = value_data.to(device).float()
        
        output = model(channel1_tensor).squeeze(1)

        loss = loss_criterion(output, value_data)
        current_loss = loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        clear_output()
        predicted_labels = (output > 0.5).float()
        correct_predictions = (predicted_labels == value_data).sum().item()
        accuracy = (correct_predictions / value_data.size(0)) * 100
        print(predicted_labels)
        print(value_data)
        print(f"Correct Predictions: {correct_predictions}")
        print(f"Total Predictions: {value_data.size(0)}")
        print(f"Accuracy: {accuracy:.4f}","%")
        print(f"At epoch: {epoch}, loss: {current_loss}")
        print(f"best accuracy: {best_accuracy}")
        
        if accuracy >= best_accuracy:
            best_accuracy = accuracy
            torch.save(model, f'E:/專題/riichi_model/CNN_riichi_model_{best_accuracy}.pth')
            print(f'模型在 epoch {epoch} 之後被儲存，正確率: {best_accuracy}')
        
        current_time = time.time()
        if current_time - last_saved_time >= 28800:
            timestamp = time.strftime("%Y%m%d_%H%M%S", time.localtime(current_time))
            model_path = f'E:/專題/riichi_model/CNN_riichi_model_{timestamp}.pth'
            torch.save(model, model_path)
            last_saved_time = current_time
            print(f'模型已基於時間間隔存儲，時間: {timestamp}')

tensor([1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0.,
        1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
       device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1.,
        0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
       device='cuda:0')
Correct Predictions: 23
Total Predictions: 32
Accuracy: 71.8750 %
At epoch: 0, loss: 0.5924042463302612
best accuracy: 96.875
